In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train_CryoSleepNull_update.csv')
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9246_01,0.0,1.0,6.0,1490.0,1.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
8689,9259_01,0.0,0.0,5.0,1893.0,0.0,2.0,44.0,0.0,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,1.0
8690,9275_03,1.0,0.0,0.0,97.0,0.0,2.0,30.0,0.0,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,1.0
8691,3896_01,1.0,0.0,2.0,140.0,1.0,0.0,18.0,0.0,0.0,4387.0,0.0,2241.0,0.0,Ainoxa Preeldy,1.0


In [4]:
df.groupby('Destination').count()
df.groupby('HomePlanet').count()
print('Cabin의 개수',len(df.groupby('Cabin').count()))

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
Destination,,,,,,,,,,,,,,,
0.0,1800,1800,1756,1750,1750,1750,1766,1757,1771,1763,1752,1761,1760,1757,1800
1.0,796,796,777,778,778,778,782,774,775,780,778,780,784,779,796
2.0,5915,5915,5765,5788,5788,5788,5787,5781,5787,5791,5778,5790,5781,5779,5915


,PassengerId,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
HomePlanet,,,,,,,,,,,,,,,
0.0,4714,4600,4616,4616,4616,4611,4626,4597,4608,4615,4598,4612,4603,4604,4714
1.0,2175,2115,2112,2112,2112,2138,2127,2133,2138,2129,2121,2130,2137,2128,2175
2.0,1804,1761,1766,1766,1766,1762,1761,1760,1766,1766,1766,1768,1765,1761,1804


Cabin의 개수 8


컬럼
- Destination 목적지 {55 Cancri e, PSO J318.5-22, TRAPPIST-1e}
- HomePlanet  {Earth , Europa, Mars}
- Cabin의 종류 6560
    - Cabin을 앞 대문자만 남기면 의미가 있을 수 있을까?
    - 돈이 많은 사람이 사는 자리 등으로 의미가 있는지확인
    - A,B,C,D,E,F,G,T로 변환 후 다시 숫자로 0~7까지 변환
- True / False 
    - CryoSleep
    - VIP
    - Transported(target)
- 삭제
    - PassengerId
    - Name

### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [3]:
# df_cabin =  df_train중에서 Cabin의 null값이 존재하는 열만 추출 
# -> 훈련된 모델의 값으로 다시 채울 예정
df_cabin = df[df['Cabin'].isnull()]

#df_train 중에서 cabin의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
df_train01 = df.dropna(subset=['Cabin'])
# df_train
df[df['Cabin'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
15,0012_01,0.0,0.0,NaN,NaN,NaN,2.0,31.0,0.0,32.0,0.0,876.0,0.0,0.0,Justie Pooles,0.0
91,0101_01,2.0,1.0,NaN,NaN,NaN,2.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,Book Trad,1.0
100,0110_01,1.0,0.0,NaN,NaN,NaN,2.0,32.0,0.0,0.0,410.0,6.0,3929.0,764.0,Graviph Aloubtled,0.0
213,0239_01,2.0,0.0,NaN,NaN,NaN,2.0,37.0,0.0,637.0,0.0,0.0,92.0,319.0,Diedow Resty,0.0
216,0244_01,2.0,1.0,NaN,NaN,NaN,2.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,Froos Sad,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8501,1116_02,0.0,0.0,NaN,NaN,NaN,2.0,36.0,0.0,0.0,3011.0,0.0,0.0,98.0,Evane Barnolaney,1.0
8503,1295_01,0.0,0.0,NaN,NaN,NaN,2.0,19.0,0.0,68.0,0.0,0.0,719.0,0.0,Nica Vaught,0.0
8619,6405_02,0.0,1.0,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Feline Toddleton,1.0
8658,7848_01,2.0,0.0,NaN,NaN,NaN,2.0,29.0,0.0,1052.0,0.0,635.0,0.0,0.0,Neypus Pine,0.0


In [4]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId','Cabin','Name'],axis=1),df_train01['Cabin'], test_size=0.2, random_state=42)

# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(6795, 13)

(6795,)

(1699, 13)

(1699,)

In [5]:
#XGBoost1
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(early_stopping_rounds=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

pred = model.predict(X_test)
acc = accuracy_score(y_test,pred)
acc

# from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

# pred = model.predict(X_test)
# # 실제 , 예측
# cm = confusion_matrix(y_test, pred)
# print('confusion_matrix : \n', cm)

# skplt.metrics.plot_confusion_matrix(y_test,pred,figsize=(8,6))
# plt.show()

# cl_report = metrics.classification_report(y_test,pred)
# print('리포트:\n',cl_report)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=10, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

0.6698057680988817

In [6]:
# df_01 은 df_train 중에서 Cabin의 값이 null인 경우의 데이터프레임
df_cabin_null = df_cabin.drop('Cabin',axis=1)
df_cabin_null

,PassengerId,HomePlanet,CryoSleep,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
15,0012_01,0.0,0.0,NaN,NaN,2.0,31.0,0.0,32.0,0.0,876.0,0.0,0.0,Justie Pooles,0.0
91,0101_01,2.0,1.0,NaN,NaN,2.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,Book Trad,1.0
100,0110_01,1.0,0.0,NaN,NaN,2.0,32.0,0.0,0.0,410.0,6.0,3929.0,764.0,Graviph Aloubtled,0.0
213,0239_01,2.0,0.0,NaN,NaN,2.0,37.0,0.0,637.0,0.0,0.0,92.0,319.0,Diedow Resty,0.0
216,0244_01,2.0,1.0,NaN,NaN,2.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,Froos Sad,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8501,1116_02,0.0,0.0,NaN,NaN,2.0,36.0,0.0,0.0,3011.0,0.0,0.0,98.0,Evane Barnolaney,1.0
8503,1295_01,0.0,0.0,NaN,NaN,2.0,19.0,0.0,68.0,0.0,0.0,719.0,0.0,Nica Vaught,0.0
8619,6405_02,0.0,1.0,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Feline Toddleton,1.0
8658,7848_01,2.0,0.0,NaN,NaN,2.0,29.0,0.0,1052.0,0.0,635.0,0.0,0.0,Neypus Pine,0.0


In [7]:
# 훈련된 xgboost모델을 사용해서 df_cabin_null의 HomePlanet값을 예측
pred = model.predict(df_cabin_null.drop(['PassengerId','Name'],axis=1))
pred

array([4, 5, 7, 3, 5, 5, 0, 4, 0, 6, 6, 0, 0, 4, 2, 3, 4, 1, 1, 5, 0, 5,
       5, 0, 5, 5, 0, 6, 4, 5, 1, 0, 4, 0, 2, 3, 5, 0, 0, 6, 1, 5, 1, 5,
       5, 5, 6, 6, 4, 3, 6, 4, 1, 7, 0, 3, 4, 1, 4, 6, 6, 6, 5, 4, 0, 0,
       4, 4, 5, 3, 4, 0, 7, 1, 7, 6, 5, 0, 5, 6, 1, 3, 7, 5, 5, 6, 5, 5,
       4, 6, 6, 2, 5, 4, 5, 4, 5, 6, 1, 0, 5, 0, 4, 6, 4, 2, 4, 6, 0, 4,
       0, 1, 5, 5, 5, 1, 6, 6, 0, 6, 6, 6, 2, 6, 6, 0, 3, 5, 6, 4, 0, 4,
       1, 4, 5, 4, 4, 3, 0, 5, 0, 3, 5, 5, 0, 3, 5, 5, 6, 0, 6, 0, 3, 6,
       0, 0, 1, 5, 6, 6, 4, 6, 1, 0, 4, 5, 6, 4, 4, 0, 6, 0, 5, 6, 5, 6,
       0, 5, 4, 6, 5, 0, 0, 0, 4, 6, 6, 5, 6, 5, 1, 0, 5, 1, 6, 4, 4, 3,
       4], dtype=int64)

In [8]:

# 예측된 값을 다시 df_cabin으로 삽입(null값 채우기)
df_cabin['Cabin'] = pred
df_cabin

C:\Users\shs03\AppData\Local\Temp\ipykernel_37588\1325591521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cabin['Cabin'] = pred


,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
15,0012_01,0.0,0.0,4,NaN,NaN,2.0,31.0,0.0,32.0,0.0,876.0,0.0,0.0,Justie Pooles,0.0
91,0101_01,2.0,1.0,5,NaN,NaN,2.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,Book Trad,1.0
100,0110_01,1.0,0.0,7,NaN,NaN,2.0,32.0,0.0,0.0,410.0,6.0,3929.0,764.0,Graviph Aloubtled,0.0
213,0239_01,2.0,0.0,3,NaN,NaN,2.0,37.0,0.0,637.0,0.0,0.0,92.0,319.0,Diedow Resty,0.0
216,0244_01,2.0,1.0,5,NaN,NaN,2.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,Froos Sad,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8501,1116_02,0.0,0.0,6,NaN,NaN,2.0,36.0,0.0,0.0,3011.0,0.0,0.0,98.0,Evane Barnolaney,1.0
8503,1295_01,0.0,0.0,4,NaN,NaN,2.0,19.0,0.0,68.0,0.0,0.0,719.0,0.0,Nica Vaught,0.0
8619,6405_02,0.0,1.0,4,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Feline Toddleton,1.0
8658,7848_01,2.0,0.0,3,NaN,NaN,2.0,29.0,0.0,1052.0,0.0,635.0,0.0,0.0,Neypus Pine,0.0


In [9]:

# null값을 채운 dataframe과 df_train01 합치기 
df_update = pd.concat([df_train01,df_cabin],axis=0)
df_update

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8501,1116_02,0.0,0.0,6.0,NaN,NaN,2.0,36.0,0.0,0.0,3011.0,0.0,0.0,98.0,Evane Barnolaney,1.0
8503,1295_01,0.0,0.0,4.0,NaN,NaN,2.0,19.0,0.0,68.0,0.0,0.0,719.0,0.0,Nica Vaught,0.0
8619,6405_02,0.0,1.0,4.0,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Feline Toddleton,1.0
8658,7848_01,2.0,0.0,3.0,NaN,NaN,2.0,29.0,0.0,1052.0,0.0,635.0,0.0,0.0,Neypus Pine,0.0


In [10]:
df_update[df_update['Cabin'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [11]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_CabinNull_update.csv', index=False)
